https://www.hackerrank.com/contests/optimization-oct17/challenges/skipping-subpath-sum

based on the discussion in
https://www.hackerrank.com/contests/optimization-oct17/challenges/skipping-subpath-sum/forum/comments/370192

## Accepted version

In [ ]:
from collections import deque

def bfs_tree(g):
    que = deque([(0, -1, 0)])
    parent = [0] * len(g)
    depth = [0] * len(g)
    visited = [False] * len(g)
    while que:
        curr, par, dep = que.popleft()
        depth[curr] = dep
        parent[curr] = par
        visited[curr] = True
        next_dep = dep + 1
        for ch in g[curr]:
            if visited[ch]:
                continue
                
            que.append((ch, curr, next_dep))
    return parent, depth

def dfs_with_depth(g, depth, parent, v, u):
    path = []
    backpath = []
    
    if depth[u] < depth[v]:
        u, v = v, u
        
    dv = depth[v]
    while depth[u] > dv:
        path.append(u)
        u = parent[u]
    
    while u != v:
        path.append(u)
        u = parent[u]
        backpath.append(v)
        v = parent[v]
    
    path.append(u)
    path += backpath[::-1]
    return path

def kadane(vals):
    max_ending_here = 0
    max_so_far = 0
    for a in vals:
        max_ending_here = max_ending_here = max(0, max_ending_here + a)
        max_so_far = max(max_so_far, max_ending_here)
    return max_so_far;

def skippingSubpathSum(n, c, graph, queries):
    answers = []
    parent, depth = bfs_tree(graph)
    for u, v in queries:
        path = dfs_with_depth(graph, depth, parent, v, u)
        odd_path = [c[n] for n in path[1::2]]
        even_path = [c[n] for n in path[::2]]
        s1 = kadane(even_path)
        s2 = kadane(odd_path)
        answers.append(max(s1, s2))
    return answers
    

## combine dfs and kadane algorithm, accepted but run a bit slower

In [ ]:
from collections import deque

def bfs_tree(g):
    que = deque([(0, -1, 0)])
    parent = [0] * len(g)
    depth = [0] * len(g)
    visited = [False] * len(g)
    while que:
        curr, par, dep = que.popleft()
        depth[curr] = dep
        parent[curr] = par
        visited[curr] = True
    
        for ch in g[curr]:
            if visited[ch]:
                continue
            que.append((ch, curr, dep + 1))
    return parent, depth

def dfs_with_depth(c, g, depth, parent, v, u):
    backpath = []
    max_ending, max_so_far = [0, 0], [0, 0]
    update_even = 0
    
    if depth[u] < depth[v]:
        u, v = v, u
        
    dv = depth[v]
    while depth[u] > dv:
        kadane_update(c[u], max_ending, max_so_far, update_even)
        update_even = 1 - update_even
        u = parent[u]
    
    while u != v:
        kadane_update(c[u], max_ending, max_so_far, update_even)
        update_even = 1 - update_even
        u = parent[u]
        backpath.append(v)
        v = parent[v]
    
    kadane_update(c[u], max_ending, max_so_far, update_even)
    update_even = 1 - update_even
    for v in  backpath[::-1]:
        kadane_update(c[v], max_ending, max_so_far, update_even)
        update_even = 1 - update_even
    return max(max_so_far)

def kadane_update(val, max_ending, max_so_far, update_even):
    max_ending[update_even] = max(0, max_ending[update_even] + val)
    max_so_far[update_even] = max(max_so_far[update_even], max_ending[update_even])

def skippingSubpathSum(n, c, graph, queries):
    answers = []
    parent, depth = bfs_tree(graph)
    for u, v in queries:
        s = dfs_with_depth(c, graph, depth, parent, v, u)
        answers.append(s)
    return answers